In [ ]:
import random, time
random.seed(42)

def rand_list(n):   return random.sample(range(10_000_000), n)
def sorted_list(n): return list(range(n))

def time_once(func, arr):
    a = arr.copy()
    t0 = time.perf_counter()
    func(a)
    return time.perf_counter() - t0

def bench(func, gen, Ns):
    return [{"N": N, "t": time_once(func, gen(N))} for N in Ns]

def show(rows, title):
    print(title)
    for r in rows:
        print(f"N={r['N']:>6}  t={r['t']:.6f}s")
    print("Ratio T(2N)/T(N):")
    for i in range(1, len(rows)):
        p, c = rows[i-1], rows[i]
        ratio = c["t"]/p["t"] if p["t"] else float("inf")
        print(f"  {p['N']:>6} → {c['N']:>6}: {ratio:.2f}")
    print("Heuristic: ~2× → n log n, ~1× → n\n")

# one-arg wrappers for the built-ins
def builtin_sorted(a):
    _ = sorted(a)   # returns a new list
    return a

def builtin_list_sort(a):
    a.sort()        # in-place
    return a


In [ ]:
Ns = [2000, 4000, 8000, 16000]

# Random data → expect ~n log n (ratios ≈ ~2×)
rows = bench(builtin_sorted,    rand_list, Ns)
show(rows, "=== built-in sorted() — Random Data ===")
rows = bench(builtin_list_sort, rand_list, Ns)
show(rows, "=== built-in list.sort() — Random Data ===")

# Already-sorted data → Timsort is adaptive → near-linear (ratios ≈ ~1×)
rows = bench(builtin_sorted,    sorted_list, Ns)
show(rows, "=== built-in sorted() — Sorted Data ===")
rows = bench(builtin_list_sort, sorted_list, Ns)
show(rows, "=== built-in list.sort() — Sorted Data ===")
